## Building Scorecards

This notebook runs through the high-level featues of building a scorecard. 

## Discretization

In [ ]:
from scorecard.scorecard import Scorecard
import seaborn as sns

df = sns.load_dataset('titanic')